In [ ]:
!pip install --upgrade grpcio grpcio-tools
!pip install "gymnasium[other]"

In [ ]:
%matplotlib inline
%load_ext tensorboard

%tensorboard --logdir ./logs_new/ --host=0.0.0.0

In [1]:
import gymnasium as gym
from gymnasium.envs.registration import register, registry
from gymnasium.wrappers import RecordVideo
import time
import numpy as np
import pygame

import matplotlib
import matplotlib.pyplot as plt

from typing import Any, Dict
import torch
import torch.nn as nn
import tensorboard

from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor

2025-02-03 16:56:55.032678: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 16:56:55.042356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738601815.054576   39169 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738601815.058371   39169 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 16:56:55.070861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='environments:MarineEnv',  # String reference to the class
    )

In [ ]:
# is_ipython = 'inline' in matplotlib.get_backend()
# if is_ipython:
#     from IPython import display

# plt.ion()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
timescale = 1 / 3
env_kwargs = dict(
    render_mode='rgb_array',
    continuous=True,
    max_episode_steps=int(400 / timescale),
    training_stage=2,
    timescale=timescale,
    training=True,
    total_targets=3,
)

In [4]:
train_env = make_vec_env(env_id='MarineEnv-v0', n_envs=8, env_kwargs=env_kwargs)
# eval_env = make_vec_env(env_id='MarineEnv-v0', n_envs=1, env_kwargs=env_kwargs)
eval_env = gym.make('MarineEnv-v0', **env_kwargs)
video_folder = './video/'
trigger = lambda x: x % 2 == 0
eval_env = RecordVideo(eval_env, video_folder, episode_trigger=trigger)

/usr/local/lib/python3.11/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /app/ProjectNew/video folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [5]:
# Linear decay from 3e-4 to 1e-4
initial_lr = 1e-3
final_lr = 1e-4
learning_rate_schedule = lambda progress_remaining: final_lr + (initial_lr - final_lr) * progress_remaining


kwargs = {
    'learning_rate': 0.0006440700528750355,
    'n_steps': 2**10,
    'batch_size': 2**10,
    'gamma': 0.9595334803327593,
    'gae_lambda': 0.9284293803518315,
    'clip_range': 0.15247146447858756,
    'ent_coef': 0.00017106771534852204,
    'vf_coef': 0.8697801969581918,
    'max_grad_norm': 1.1421017563147962,
    'target_kl': 0.19795582328410327,
    'n_epochs': 6,
    
    # 'clip_range': 0.2,  # Reduce to prevent large updates
    # 'ent_coef': 5e-2,  # Higher entropy to encourage exploration
    # 'gamma': 0.99, 
    # 'learning_rate': learning_rate_schedule,
    # 'n_steps': 2048,  # Increase from default (512) to 2048
    # 'batch_size': 512,  # Adjust batch size for stability
    # 'gae_lambda': 0.95,  # Generalized Advantage Estimation smoothing
    # 'max_grad_norm': 0.9, 
    'device': 'cpu',
    'tensorboard_log': './logs_new/',
    'policy_kwargs': {'net_arch': [128, 128], 'activation_fn': torch.nn.Tanh},  # Slightly deeper network
}

In [6]:
# Create the evaluation callback
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path='./logs_new/best_model/',
    log_path='./logs_new/results/',
    eval_freq=5000,
    deterministic=True,
    render=False
)

In [7]:
agent = PPO(
    policy='MlpPolicy',
    env=train_env,
    verbose=0,
    **kwargs
)

In [ ]:
agent.learn(total_timesteps=(2e5), reset_num_timesteps=False, progress_bar=True, tb_log_name='ppo_9', callback=eval_callback)

Output()

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation 
environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and 
rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(

Eval num_timesteps=40000, episode_reward=-141.14 +/- 843.71

Episode length: 163.20 +/- 62.85

New best mean reward!

...

Episode length: 178.80 +/- 85.94

New best mean reward!

Eval num_timesteps=120000, episode_reward=759.07 +/- 230.83

Episode length: 223.80 +/- 79.28

New best mean reward!

In [ ]:
best_agent = agent.load('./logs_new/best_model/best_model.zip', device='cpu')

In [ ]:
eval_env = gym.make('MarineEnv-v0', **env_kwargs)
mean, std = evaluate_policy(model=best_agent, env=eval_env, n_eval_episodes=10, deterministic=True)
print(f'Mean: {mean:.2f}, Std: {std:.2f}')

In [ ]:
# Save environment normalization stats
# env.save("ppo_normalized_env.pkl")
# agent.save("ppo")
best_agent.save('ppo')
# agent = agent.load("ppo", device='cpu')
# model = model.load('ppo_marine_stage_1.zip')

In [ ]:
env = VecNormalize.load("ppo_normalized_env.pkl", env)

# Disable reward normalization for evaluation
env.training = False
env.norm_reward = False

import cv2
import numpy as np

obs = env.reset()
for _ in range(100):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, dones, _ = env.step(action)

    # ✅ Ensure env.get_images() is not empty
    images = env.get_images()
    if images and images[0] is not None:
        frame = images[0]
        
        # ✅ Ensure the frame has valid dimensions before displaying
        if frame.shape[0] > 0 and frame.shape[1] > 0:
            cv2.imshow("PPO MarineEnv Evaluation", frame)
            cv2.waitKey(1)  # Display for 1ms
        else:
            print("Warning: Received an empty frame from env.get_images()")

    if dones:
        break

env.close()
cv2.destroyAllWindows()  # Close display window


In [ ]:
timescale = 1/6
env_trn = gym.make('MarineEnv-v0', render_mode='human', continuous=True, training_stage=2, timescale=timescale, training=False, total_targets=3)
for _ in range(5):
    
    state, _ = env_trn.reset()
    print(f'Detected targets:', [target for target in env_trn.unwrapped.own_ship.detected_targets])
    print(state)
    episode_rewards = 0 
    # flatten_state = flatten(env.observation_space, state)
    # state = torch.tensor(flatten_state, dtype=torch.float32, device=device).unsqueeze(0)
    for _ in range(int(400 / timescale)):
        action = best_agent.predict(state, deterministic=True)
        # action = agent.predict(state, deterministic=True)
        # print(action)
        # observation, reward, terminated, truncated, info = env_trn.step((0, 0))
        observation, reward, terminated, truncated, info = env_trn.step(action[0])
        env_trn.render()
        time.sleep(0.005)
        episode_rewards += reward
        print('===========================')
        print(observation)
        print(f'Step reward: {reward:.2f}')
        print(f'Current Total reward: {episode_rewards:.2f}')
        print(f'Dangerous targets: ', [target for target in env_trn.unwrapped.own_ship.dangerous_targets])
        
        if terminated or truncated:
            print('Episode total reward: ', episode_rewards)
            print(info)
            break
    
        state = observation
            
    print('Episode total rewards: ', episode_rewards)
    print('Episode final state: ', state)
    print(f'============================\n' * 10)
    env_trn.close()

In [ ]:
import os
from gymnasium.wrappers import RecordVideo

from IPython.display import HTML
from base64 import b64encode

# Create a folder to save videos
video_folder = './video'
os.makedirs(video_folder, exist_ok=True)

env_trn = gym.make('MarineEnv-v0', render_mode='rgb_array', continuous=True, training_stage=2, timescale=timescale, training=False, total_targets=3)

# Wrap the environment
env = RecordVideo(env_trn, video_folder)

# Run a single episode to record
for _ in range(5):
    state, _ = env.reset()
    for _ in range(int(400 / timescale)):
            action = best_agent.predict(state, deterministic=True)
            observation, reward, terminated, truncated, info = env.step(action[0])
            time.sleep(0.005)
        
            if terminated or truncated:
                break
       
            state = observation

env.close()

# Locate the video file (the Monitor wrapper saves it with an .mp4 extension)
import glob
video_files = glob.glob(video_folder + "/*.mp4")
print("Recorded video files:", video_files)



In [ ]:
# Display the first recorded video (if available)
if video_files:
    video_file = video_files[0]
    video = open(video_file, "rb").read()
    video_b64 = b64encode(video).decode("utf-8")
    HTML(f"""
    <video width="640" height="480" controls>
      <source src="data:video/mp4;base64,{video_b64}" type="video/mp4">
    </video>
    """)